In [ ]:
!nvidia-smi

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.utils.data as data
from tqdm import tqdm

In [ ]:
device = torch.device('cuda')
model = torchvision.models.resnet101(pretrained=True)

for param in model.parameters():
    param.requires_grad = False

model.fc = torch.nn.Linear(model.fc.in_features, 17)
model = model.to(device)
input_size = 224

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
num_epochs = 16
batch_size = 16
lr = 0.001

TRAIN_DATA_PATH = "/content/drive/My Drive/datasets/Flowers/Train/"

TRANSFORM_IMG = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(input_size),
    transforms.ToTensor(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225] )
    ])

train_data = torchvision.datasets.ImageFolder(root=TRAIN_DATA_PATH, transform=TRANSFORM_IMG)
train_data_loader = data.DataLoader(train_data, batch_size=batch_size, shuffle=True,  num_workers=4)

In [ ]:
# Loss and Optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [ ]:
print(model)

In [ ]:
# Train the Model
for epoch in range(num_epochs):
    for (images, labels) in tqdm(train_data_loader):
        # Forward + Backward + Optimize
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.data.item()}')

In [ ]:
torch.cuda.empty_cache()

In [ ]:
batch_size = 8

TEST_DATA_PATH = "/content/drive/My Drive/datasets/Flowers/Test/"

TRANSFORM_IMG = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(input_size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225] )
    ])

test_data = torchvision.datasets.ImageFolder(root=TEST_DATA_PATH, transform=TRANSFORM_IMG)
test_data_loader  = data.DataLoader(test_data, batch_size=batch_size, shuffle=True, num_workers=4) 

In [ ]:
# Test the Model
model.eval()  # Change model to 'eval' mode (BN uses moving mean/var)
correct = 0
total = 0
for images, labels in tqdm(test_data_loader):

    images = images.to(device)
    labels = labels.to(device)
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()

print('Test Accuracy:', (100.0*correct/total))

In [ ]:
#Save the Trained Model
torch.save(model.state_dict(), 'flowers.pth')